# Installation

In [7]:
!pip3 install ultralytics

Defaulting to user installation because normal site-packages is not writeable


# Extract .zip
> Don't forget to import .zip into Colab !!!

In [8]:
import zipfile

def ext_zip(filename, extract_path):

    if os.path.exists(extract_path):
        print(f"📁 {extract_path} already exists !!!")
        return

    print("📦 Unzipping...")
    with zipfile.ZipFile(filename, 'r') as zip_ref:
        zip_ref.extractall(extract_path)

    print(f"✅ Extracted to: {extract_path}")

# Train model

## "dataset_1"
> https://universe.roboflow.com/roboflow-100/chess-pieces-mjzgj/dataset/2

### Import & Extract

In [1]:
# ใช้ %pip เพื่อให้มั่นใจว่าติดตั้งลงใน Kernel ปัจจุบันที่ใช้อยู่
%pip install PyYAML ultralytics torch torchvision IPython

Note: you may need to restart the kernel to use updated packages.


In [2]:
"""
Chess Piece Detection - แปลง Labels และ Train YOLOv8
สำหรับ dataset ที่โหลดมาแล้ว
"""

import os
import shutil
import yaml
from pathlib import Path
from ultralytics import YOLO
import torch
from IPython.display import Image as IPImage, display

# ===== Extract Dataset File =====
# ext_zip("dataset_1.zip", "dataset_1")

### Configuration

In [3]:
# ===== Configuration =====
class Config:
    # ระบุ path ของ dataset ที่โหลดมา
    DATASET_DIR = "dataset_1"  # แก้ตามชื่อ folder จริง

    # Class Mapping: จาก dataset เดิม -> class ใหม่ที่ต้องการ
    # Dataset เดิมมี: ['bishop', 'black-bishop', 'black-king', 'black-knight',
    #                   'black-pawn', 'black-queen', 'black-rook', 'white-bishop',
    #                   'white-king', 'white-knight', 'white-pawn', 'white-queen', 'white-rook']

    OLD_TO_NEW_CLASS = {
        0: -1,  # 'bishop' -> ลบทิ้ง (ไม่ระบุสี)
        1: 0,   # 'black-bishop' -> 0
        2: 1,   # 'black-king' -> 1
        3: 2,   # 'black-knight' -> 2
        4: 3,   # 'black-pawn' -> 3
        5: 4,   # 'black-queen' -> 4
        6: 5,   # 'black-rook' -> 5
        7: 6,   # 'white-bishop' -> 6
        8: 7,   # 'white-king' -> 7
        9: 8,   # 'white-knight' -> 8
        10: 9,  # 'white-pawn' -> 9
        11: 10, # 'white-queen' -> 10
        12: 11  # 'white-rook' -> 11
    }

    NEW_CLASS_NAMES = [
        "black-bishop", "black-king", "black-knight", "black-pawn",
        "black-queen", "black-rook", "white-bishop", "white-king",
        "white-knight", "white-pawn", "white-queen", "white-rook"
    ]

    # Training Parameters
    MODEL_SIZE = "yolov8x.pt"  # n, s, m, l, x
    EPOCHS = 100
    IMG_SIZE = 640
    BATCH_SIZE = 16


def convert_all_labels():
    """แปลง class labels ทั้งหมดในทุก split"""
    print("🔄 Converting class labels...")

    splits = ['train', 'valid', 'test']
    total_converted = 0

    for split in splits:
        label_dir = os.path.join(Config.DATASET_DIR, split, 'labels')

        if not os.path.exists(label_dir):
            print(f"  ⚠️  {split}/labels not found, skipping...")
            continue

        converted_count = 0
        label_files = [f for f in os.listdir(label_dir) if f.endswith('.txt')]

        for label_file in label_files:
            label_path = os.path.join(label_dir, label_file)

            # อ่าน labels
            try:
                with open(label_path, 'r') as f:
                    lines = f.readlines()
            except:
                continue

            # แปลง class
            new_lines = []
            for line in lines:
                parts = line.strip().split()
                if len(parts) < 5:
                    continue

                try:
                    old_class = int(parts[0])
                except:
                    continue

                # แปลง class
                if old_class in Config.OLD_TO_NEW_CLASS:
                    new_class = Config.OLD_TO_NEW_CLASS[old_class]

                    # ข้ามถ้าเป็น -1 (ไม่ต้องการ class นี้)
                    if new_class == -1:
                        continue

                    new_line = f"{new_class} {' '.join(parts[1:])}\n"
                    new_lines.append(new_line)

            # บันทึก labels ใหม่
            with open(label_path, 'w') as f:
                f.writelines(new_lines)

            converted_count += 1

        print(f"  ✅ {split}: {converted_count} files converted")
        total_converted += converted_count

    print(f"\n✅ Total: {total_converted} label files converted!")
    return total_converted > 0


def create_new_yaml():
    """สร้าง data.yaml ใหม่"""
    print("\n📝 Creating new data.yaml...")

    yaml_content = {
        'path': os.path.abspath(Config.DATASET_DIR),
        'train': 'train/images',
        'val': 'valid/images',
        'test': 'test/images',
        'nc': len(Config.NEW_CLASS_NAMES),
        'names': Config.NEW_CLASS_NAMES
    }

    yaml_path = os.path.join(Config.DATASET_DIR, 'data_new.yaml')

    with open(yaml_path, 'w') as f:
        yaml.dump(yaml_content, f, default_flow_style=False, sort_keys=False)

    print(f"✅ Created: {yaml_path}")
    print(f"\nNew classes ({len(Config.NEW_CLASS_NAMES)}):")
    for i, name in enumerate(Config.NEW_CLASS_NAMES):
        print(f"  {i}: {name}")

    return yaml_path


def train_model(yaml_path):
    """Train YOLOv8 with augmentation"""
    print("\n" + "=" * 60)
    print("🚀 Starting Training")
    print("=" * 60)

    # Detect Device
    device = 'cpu'
    if torch.cuda.is_available():
        device = 0
        print(f"🎮 GPU: {torch.cuda.get_device_name(0)}")
    elif torch.backends.mps.is_available():
        device = 'mps'
        print("🍎 Mac GPU (MPS) Detected")

    # Load model
    model = YOLO(Config.MODEL_SIZE)
    print(f"Model: {Config.MODEL_SIZE}")
    print(f"Epochs: {Config.EPOCHS}")
    print(f"Image size: {Config.IMG_SIZE}")
    print(f"Batch size: {Config.BATCH_SIZE}")
    print("=" * 60 + "\n")

    # Train with STRONG augmentation
    results = model.train(
        data=yaml_path,
        epochs=Config.EPOCHS,
        imgsz=Config.IMG_SIZE,
        batch=Config.BATCH_SIZE,

        # ===== Geometric Augmentation =====
        degrees=5.0,          # Rotation ±5°
        translate=0.05,         # Translation ±5%
        scale=0.1,             # Scale 10%
        shear=2.0,             # Shear ±2°
        perspective=0.0001,    # Perspective transform
        flipud=0.0,            # No vertical flip (chess oriented)
        fliplr=0.4,            # Horizontal flip 40%

        # ===== Color Augmentation =====
        hsv_h=0.005,           # Hue shift ±0.5%
        hsv_s=0.2,             # Saturation ±20%
        hsv_v=0.1,             # Value/Brightness ±10%

        # ===== Advanced Augmentation =====
        mosaic=0.2,            # Mosaic 4-image mix 100%
        mixup=0.0,             # MixUp augmentation 0%
        copy_paste=0.0,        # Copy-paste augmentation 0%

        # ===== Training Settings =====
        patience=50,           # Early stopping patience
        save=True,             # Save checkpoints
        device=device,         # GPU or CPU
        workers=4 if device != 'cpu' else 2,
        project='runs/chess_detection',
        name='exp',
        exist_ok=True,
        pretrained=True,
        optimizer='auto',      # AdamW or SGD
        verbose=True,
        seed=42,
        deterministic=True,

        # ===== Performance =====
        amp=True if device != 'cpu' else False,  # Mixed precision
        fraction=1.0,          # Use 100% of data
        cache=False,           # Don't cache (save RAM)

        # ===== Validation =====
        val=True,
        plots=True,
        save_period=-1,        # Save every N epochs (-1 = only best/last)
    )

    print("\n" + "=" * 60)
    print("✅ Training Completed!")
    print("=" * 60)

    return results


def validate_model(model_path):
    """Validate the trained model"""
    print("\n🔍 Validating model...")

    model = YOLO(model_path)
    metrics = model.val()

    print("\n📊 Validation Results:")
    print("=" * 40)
    print(f"  mAP50:     {metrics.box.map50:.4f}")
    print(f"  mAP50-95:  {metrics.box.map:.4f}")
    print(f"  Precision: {metrics.box.mp:.4f}")
    print(f"  Recall:    {metrics.box.mr:.4f}")
    print("=" * 40)

    return metrics


def test_prediction(model_path, test_image_path):
    """Test prediction on a single image"""
    if not os.path.exists(test_image_path):
        return

    print(f"\n🎯 Testing: {os.path.basename(test_image_path)}")

    model = YOLO(model_path)
    results = model.predict(
        source=test_image_path,
        save=True,
        conf=0.25,
        iou=0.3,
        show_labels=True,
        show_conf=True,
        line_width=2
    )

    # Display result in Colab
    save_path = results[0].save_dir
    pred_img = os.path.join(save_path, os.path.basename(test_image_path))

    if os.path.exists(pred_img):
        print("📸 Prediction Result:")
        display(IPImage(filename=pred_img, width=600))


def show_training_plots():
    """Display training plots"""
    print("\n📊 Training Visualizations")
    print("=" * 60)

    plots = {
        'Training Results': 'runs/chess_detection/exp/results.png',
        'Confusion Matrix': 'runs/chess_detection/exp/confusion_matrix.png',
        'Validation Predictions': 'runs/chess_detection/exp/val_batch0_pred.jpg',
        'F1 Curve': 'runs/chess_detection/exp/F1_curve.png',
        'PR Curve': 'runs/chess_detection/exp/PR_curve.png',
    }

    for title, plot_path in plots.items():
        if os.path.exists(plot_path):
            print(f"\n📈 {title}:")
            display(IPImage(filename=plot_path, width=800))

    print("=" * 60)



def main():
    """Main execution"""
    print("=" * 60)
    print("♟️  Chess Piece Detection - YOLOv8 Training")
    print("=" * 60)

    # Check GPU
    if torch.cuda.is_available():
        print(f"🎮 GPU: {torch.cuda.get_device_name(0)}")
        print(f"💾 Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
    else:
        print("⚠️  No GPU detected!")
        print("   Training on CPU will be VERY SLOW (8-12 hours)")
        print("   Enable GPU: Runtime > Change runtime type > T4 GPU")

    print("=" * 60 + "\n")

    # Step 1: Convert labels
    if not convert_all_labels():
        print("\n❌ Failed to convert labels!")
        return

    # Step 2: Create new yaml
    yaml_path = create_new_yaml()

    # Step 3: Train
    train_model(yaml_path)

    # Step 4: Validate
    best_model = "runs/chess_detection/exp/weights/best.pt"

    if os.path.exists(best_model):
        # Validate
        validate_model(best_model)

        # Show plots
        show_training_plots()

        # Test predictions
        print("\n🎯 Testing Predictions")
        print("=" * 60)

        valid_img_dir = os.path.join(Config.DATASET_DIR, 'valid', 'images')
        if os.path.exists(valid_img_dir):
            test_imgs = [f for f in os.listdir(valid_img_dir)
                        if f.endswith(('.jpg', '.png', '.jpeg'))][:3]

            for img_name in test_imgs:
                test_prediction(best_model, os.path.join(valid_img_dir, img_name))

        print("\n" + "=" * 60)
        print("✨ All Done! Your model is ready!")
        print("=" * 60)
        print(f"\n📁 Best model: {best_model}")
        print(f"📁 Last model: runs/chess_detection/exp/weights/last.pt")
        print("\n💡 To download your model:")
        print("   from google.colab import files")
        print(f"   files.download('{best_model}')")
        print("=" * 60)
    else:
        print(f"\n❌ Model not found: {best_model}")


def download_model():
    """Download the trained model"""
    from google.colab import files

    best_model = "runs/chess_detection/exp/weights/best.pt"
    last_model = "runs/chess_detection/exp/weights/last.pt"

    if os.path.exists(best_model):
        print("📥 Downloading best.pt...")
        files.download(best_model)
        print("✅ best.pt downloaded!")

    if os.path.exists(last_model):
        print("📥 Downloading last.pt...")
        files.download(last_model)
        print("✅ last.pt downloaded!")

### Run

In [4]:
# ===== Run Pipeline =====
if __name__ == "__main__":
    main()

# After training, download model:
# download_model()

♟️  Chess Piece Detection - YOLOv8 Training
⚠️  No GPU detected!
   Training on CPU will be VERY SLOW (8-12 hours)
   Enable GPU: Runtime > Change runtime type > T4 GPU

🔄 Converting class labels...
  ✅ train: 202 files converted
  ✅ valid: 58 files converted
  ✅ test: 29 files converted

✅ Total: 289 label files converted!

📝 Creating new data.yaml...
✅ Created: dataset_1/data_new.yaml

New classes (12):
  0: black-bishop
  1: black-king
  2: black-knight
  3: black-pawn
  4: black-queen
  5: black-rook
  6: white-bishop
  7: white-king
  8: white-knight
  9: white-pawn
  10: white-queen
  11: white-rook

🚀 Starting Training
🍎 Mac GPU (MPS) Detected
Model: yolov8x.pt
Epochs: 100
Image size: 640
Batch size: 16

Ultralytics 8.3.235 🚀 Python-3.10.19 torch-2.9.1 MPS (Apple M4)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=

RuntimeError: MPS backend out of memory (MPS allocated: 17.64 GiB, other allocations: 502.92 MiB, max allowed: 18.13 GiB). Tried to allocate 256 bytes on shared pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).

## "dataset_2"
> https://drive.google.com/drive/folders/1kNlKrD02TvXmzp2fs1FZD6eNe_0RvUKd

In [ ]:
"""
Fine-tune YOLOv8 ด้วย Real Chess Board Dataset
Dataset มี YOLO labels อยู่แล้ว แค่ต้องแปลง class mapping
"""

import os
import shutil
from pathlib import Path
import yaml
from ultralytics import YOLO
import torch
from IPython.display import Image as IPImage, display
import random

# ===== Extract Dataset File =====
# ext_zip("dataset_2.zip", "dataset_2")

# ===== Configuration =====
class Config:
    # ระบุ path ของ dataset
    INPUT_DIR = "dataset_2"  # folder หลัก
    OUTPUT_DIR = "dataset_2_restruct_AJ_8020"

    # Class mapping จาก dataset เดิม -> class ใหม่ของเรา
    # Dataset เดิม:
    OLD_CLASSES = {
        0: 'WhiteQueen',
        1: 'WhitePawn',
        2: 'BlackRook',
        3: 'BlackBishop',
        4: 'BlackKnight',
        5: 'BlackQueen',
        6: 'BlackPawn',
        7: 'Blackking',      # typo ในชื่อ
        8: 'WhiteRook',
        9: 'WhiteBishop',
        10: 'WhiteKnight',
        11: 'WhiteKing'
    }

    # Class ของเรา (ตามลำดับที่ต้องการ)
    NEW_CLASS_NAMES = [
        "black-bishop", "black-king", "black-knight", "black-pawn",
        "black-queen", "black-rook", "white-bishop", "white-king",
        "white-knight", "white-pawn", "white-queen", "white-rook"
    ]

    # Mapping: OLD class_id -> NEW class_id
    OLD_TO_NEW = {
        0: 10,  # WhiteQueen -> white-queen
        1: 9,   # WhitePawn -> white-pawn
        2: 5,   # BlackRook -> black-rook
        3: 0,   # BlackBishop -> black-bishop
        4: 2,   # BlackKnight -> black-knight
        5: 4,   # BlackQueen -> black-queen
        6: 3,   # BlackPawn -> black-pawn
        7: 1,   # Blackking -> black-king
        8: 11,  # WhiteRook -> white-rook
        9: 6,   # WhiteBishop -> white-bishop
        10: 8,  # WhiteKnight -> white-knight
        11: 7   # WhiteKing -> white-king
    }

    # Pre-trained model
    PRETRAINED_MODEL = "runs/chess_detection/exp/weights/best.pt"

    # Training
    EPOCHS = 150
    IMG_SIZE = 640  # full board ใช้ 640
    BATCH_SIZE = 16
    TRAIN_VAL_SPLIT = 0.80  # 80% train, 20% val
    LR0 = 0.001  
    LRF = 0.01
    CLS_W = 1.0   # Classification Loss
    BOX_W = 7.5   # Bounding Box Loss
    PATIENCE = 100 


def convert_labels():
    """แปลง class IDs ใน label files (รองรับทั้งรูปเดี่ยวและ full board)"""
    print("🔄 Converting label files...")

    # ลบ output เก่า
    if os.path.exists(Config.OUTPUT_DIR):
        shutil.rmtree(Config.OUTPUT_DIR)

    # สร้าง structure
    os.makedirs(f"{Config.OUTPUT_DIR}/images/train", exist_ok=True)
    os.makedirs(f"{Config.OUTPUT_DIR}/images/val", exist_ok=True)
    os.makedirs(f"{Config.OUTPUT_DIR}/labels/train", exist_ok=True)
    os.makedirs(f"{Config.OUTPUT_DIR}/labels/val", exist_ok=True)

    # หา Images และ Label folders
    images_dir = os.path.join(Config.INPUT_DIR, 'Images')
    labels_dir = os.path.join(Config.INPUT_DIR, 'Label')

    if not os.path.exists(images_dir) or not os.path.exists(labels_dir):
        print(f"❌ Error: Images or Label folder not found!")
        print(f"   Looking for: {images_dir}")
        print(f"              : {labels_dir}")
        return False

    # อ่านไฟล์ทั้งหมด
    image_files = [f for f in os.listdir(images_dir)
                   if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

    print(f"📊 Found {len(image_files)} images")

    # แยกประเภทรูป
    single_piece_images = []
    full_board_images = []

    for img_file in image_files:
        # ตรวจสอบจากชื่อว่าเป็นรูปอะไร
        lower_name = img_file.lower()

        # ถ้าชื่อมี "fullboard" หรือ "board" หรือ "game" = full board
        if any(keyword in lower_name for keyword in ['fullboard', 'board', 'game', 'cp']):
            full_board_images.append(img_file)
        else:
            single_piece_images.append(img_file)

    print(f"\n📋 Dataset breakdown:")
    print(f"   Single pieces: {len(single_piece_images)}")
    print(f"   Full boards: {len(full_board_images)}")
    print(f"   Total: {len(image_files)}")

    # Shuffle เพื่อแบ่ง train/val แบบสุ่ม
    random.shuffle(single_piece_images)
    random.shuffle(full_board_images)

    converted_train = 0
    converted_val = 0
    skipped = 0

    # Process ทั้งหมด
    all_images = single_piece_images + full_board_images
    train_count = int(len(all_images) * Config.TRAIN_VAL_SPLIT)

    print(f"\n🔀 Split: {train_count} train ({Config.TRAIN_VAL_SPLIT*100:.0f}%), "
          f"{len(all_images) - train_count} val ({(1-Config.TRAIN_VAL_SPLIT)*100:.0f}%)")

    for idx, img_file in enumerate(all_images):
        # แบ่ง train/val ตามลำดับ
        is_val = idx >= train_count
        split = 'val' if is_val else 'train'

        # หา label file
        base_name = os.path.splitext(img_file)[0]
        label_file = base_name + '.txt'

        img_path = os.path.join(images_dir, img_file)
        label_path = os.path.join(labels_dir, label_file)

        # ตรวจสอบว่ามี label หรือไม่
        if not os.path.exists(label_path):
            print(f"  ⚠️  No label for: {img_file}")
            skipped += 1
            continue

        # Copy image
        shutil.copy(img_path, f"{Config.OUTPUT_DIR}/images/{split}/{img_file}")

        # แปลง และ copy label
        try:
            with open(label_path, 'r') as f:
                lines = f.readlines()

            new_lines = []
            objects_count = 0

            for line in lines:
                parts = line.strip().split()
                if len(parts) < 5:
                    continue

                old_class = int(parts[0])

                # แปลง class
                if old_class not in Config.OLD_TO_NEW:
                    print(f"  ⚠️  Unknown class {old_class} in {img_file}")
                    continue

                new_class = Config.OLD_TO_NEW[old_class]

                # สร้าง line ใหม่
                new_line = f"{new_class} {' '.join(parts[1:])}\n"
                new_lines.append(new_line)
                objects_count += 1

            # บันทึก label ใหม่
            new_label_path = f"{Config.OUTPUT_DIR}/labels/{split}/{base_name}.txt"
            with open(new_label_path, 'w') as f:
                f.writelines(new_lines)

            if is_val:
                converted_val += 1
            else:
                converted_train += 1

            # แสดง progress
            if (idx + 1) % 100 == 0:
                print(f"  ✓ Processed {idx + 1}/{len(all_images)} images")

        except Exception as e:
            print(f"  ❌ Error processing {img_file}: {e}")
            skipped += 1
            continue

    print(f"\n✅ Conversion complete!")
    print(f"   Train: {converted_train} images")
    print(f"   Val: {converted_val} images")
    print(f"   Skipped: {skipped} images")

    return converted_train > 0


def analyze_dataset():
    """วิเคราะห์ dataset หลังแปลง"""
    print("\n📊 Analyzing converted dataset...")

    from collections import Counter

    class_counts = Counter()

    for split in ['train', 'val']:
        label_dir = f"{Config.OUTPUT_DIR}/labels/{split}"

        if not os.path.exists(label_dir):
            continue

        for label_file in os.listdir(label_dir):
            if not label_file.endswith('.txt'):
                continue

            label_path = os.path.join(label_dir, label_file)

            with open(label_path, 'r') as f:
                for line in f:
                    parts = line.strip().split()
                    if len(parts) >= 5:
                        class_id = int(parts[0])
                        class_counts[class_id] += 1

    print("\n📈 Class Distribution:")
    print("=" * 60)

    total = sum(class_counts.values())

    for class_id in range(len(Config.NEW_CLASS_NAMES)):
        count = class_counts.get(class_id, 0)
        percentage = (count / total * 100) if total > 0 else 0
        bar = "█" * int(percentage / 2)

        print(f"{class_id:2d}. {Config.NEW_CLASS_NAMES[class_id]:15s} | "
              f"{count:5d} ({percentage:5.1f}%) {bar}")

    print("=" * 60)
    print(f"Total objects: {total}")


def create_yaml():
    """สร้าง data.yaml"""
    print("\n📝 Creating data.yaml...")

    yaml_content = {
        'path': os.path.abspath(Config.OUTPUT_DIR),
        'train': 'images/train',
        'val': 'images/val',
        'nc': len(Config.NEW_CLASS_NAMES),
        'names': Config.NEW_CLASS_NAMES
    }

    yaml_path = f"{Config.OUTPUT_DIR}/data.yaml"

    with open(yaml_path, 'w') as f:
        yaml.dump(yaml_content, f, default_flow_style=False, sort_keys=False)

    print(f"✅ Created: {yaml_path}")
    return yaml_path


def fine_tune_model(yaml_path):
    """Fine-tune model กับ real board data"""
    print("\n" + "=" * 60)
    print("🔥 Fine-tuning with Real Chess Board Data")
    print("=" * 60)

    # Check GPU
    device = 'cpu'
    if torch.cuda.is_available():
        device = 0
        print(f"🎮 GPU DETECTED: {torch.cuda.get_device_name(0)}")
        print(f"💾 VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
    elif torch.backends.mps.is_available():
        device = 'mps'
        print(f"🍎 Mac GPU (MPS) DETECTED!")
        print("🚀 Running on Apple Silicon Metal Performance Shaders")
    else:
        print("\n⚠️  Training on CPU will be slow!")
        # ถ้าอยากบังคับรัน 150 epoch บน CPU ให้ comment 2 บรรทัดล่างนี้ทิ้งครับ
        Config.EPOCHS = 30
        Config.BATCH_SIZE = 4

    # Load pre-trained model
    if os.path.exists(Config.PRETRAINED_MODEL):
        print(f"\n✅ Loading pre-trained model: {Config.PRETRAINED_MODEL}")
        model = YOLO(Config.PRETRAINED_MODEL)
    else:
        print(f"\n⚠️  Pre-trained model not found!")
        print(f"   Using YOLOv8n instead...")
        model = YOLO('yolov8n.pt')

    print(f"\n📋 Training Configuration:")
    print(f"   Epochs: {Config.EPOCHS}")
    print(f"   Batch size: {Config.BATCH_SIZE}")
    print(f"   Image size: {Config.IMG_SIZE}")
    print("=" * 60 + "\n")

    # Fine-tune with adjusted augmentation and learning rates
    results = model.train(
        data=yaml_path,
        epochs=Config.EPOCHS,
        imgsz=Config.IMG_SIZE,
        batch=Config.BATCH_SIZE,

        # Geometric augmentation (ลดลงสำหรับภาพจริง)
        degrees=5.0,        # ลดการหมุน
        translate=0.05,     # ลดการเลื่อน
        scale=0.2,          # ลดช่วง scale
        shear=0.0,          # ปิด shear
        perspective=0.0,    # ปิด perspective warping
        flipud=0.0,         
        fliplr=0.5,   

        degrees=10,                 # ลดการหมุน
        translate=0.10,             # ลดการเลื่อน
        scale=0.15,                 # ลดช่วง scale
        shear=2.0,                  
        perspective=0.0,
        flipud=0.0,
        fliplr=0.5,   

        # Color augmentation (ปรับลด Saturation/Brightness)
        hsv_h=0.015,
        hsv_s=0.50,
        hsv_v=0.30,
        
        # Advanced augmentation (ปิด Copy/Paste)
        mosaic=0.5,
        mixup=0.05,
        copy_paste=0.0,

        # Optimization
        momentum=0.937,
        weight_decay=0.0005,
        warmup_epochs=3.0,
        warmup_momentum=0.8,

        # Training settings
        patience=Config.PATIENCE, # เพิ่ม Patience
        save=True,
        device=device,
        workers=4 if device != 'cpu' else 2,
        project='runs/chess_board_finetuned_v2_100epochs_8020', # เปลี่ยนชื่อ project เพื่อไม่ทับของเดิม
        name='exp_100epoch_dataset3',
        exist_ok=True,
        
        # Loss settings (เพิ่มน้ำหนักการจำแนกคลาส)
        cls=Config.CLS_W,
        box=Config.BOX_W,
        dfl=1.5,

        # Learning rate (ต่ำสำหรับ Fine-tune)
        lr0=Config.LR0,
        lrf=Config.LRF,

        amp=True if device != 'cpu' else False,
        fraction=1.0,
        cache=False,
    )

    print("\n✅ Fine-tuning completed!")
    return results


def validate_model(model_path):
    """Validate model"""
    print("\n🔍 Validating model...")

    model = YOLO(model_path)
    metrics = model.val()

    print("\n📊 Validation Results:")
    print("=" * 40)
    print(f"  mAP50:     {metrics.box.map50:.4f}")
    print(f"  mAP50-95:  {metrics.box.map:.4f}")
    print(f"  Precision: {metrics.box.mp:.4f}")
    print(f"  Recall:    {metrics.box.mr:.4f}")
    print("=" * 40)

    return metrics


def test_predictions(model_path):
    """ทดสอบ predictions"""
    print("\n🎯 Testing predictions...")

    model = YOLO(model_path)

    val_img_dir = f"{Config.OUTPUT_DIR}/images/val"

    if not os.path.exists(val_img_dir):
        print("  ⚠️  Validation images not found")
        return

    test_imgs = [f for f in os.listdir(val_img_dir)
                 if f.endswith(('.jpg', '.png'))][:3]

    for img_name in test_imgs:
        img_path = os.path.join(val_img_dir, img_name)

        results = model.predict(
            source=img_path,
            save=True,
            conf=0.15,         # threshold ต่ำสำหรับหมากดำ
            iou=0.3,
            show_labels=True,
            show_conf=True,
            line_width=2
        )

        # Display result
        save_path = results[0].save_dir
        pred_img = os.path.join(save_path, img_name)

        if os.path.exists(pred_img):
            print(f"\n📸 {img_name}:")
            display(IPImage(filename=pred_img, width=800))


def show_training_plots():
    """แสดง training plots"""
    plots = [
        'runs/chess_board_finetuned/exp/results.png',
        'runs/chess_board_finetuned/exp/confusion_matrix.png',
        'runs/chess_board_finetuned/exp/val_batch0_pred.jpg'
    ]

    print("\n📊 Training Visualizations:")
    for plot in plots:
        if os.path.exists(plot):
            display(IPImage(filename=plot, width=900))


def main():
    """Main execution"""
    print("=" * 60)
    print("♟️  Real Chess Board Dataset Fine-tuning")
    print("=" * 60)

    # Check GPU
    if torch.cuda.is_available():
        print(f"🎮 GPU: {torch.cuda.get_device_name(0)}")
    else:
        print("⚠️  No GPU! Enable GPU for faster training")

    print("=" * 60 + "\n")

    # Step 1: Convert labels
    success = convert_labels()

    if not success:
        print("\n❌ Failed to convert labels!")
        return

    # Step 2: Analyze dataset
    analyze_dataset()

    # Step 3: Create yaml
    yaml_path = create_yaml()

    # Step 4: Fine-tune
    fine_tune_model(yaml_path)

    # Step 5: Validate
    best_model = "runs/chess_board_finetuned/exp/weights/best.pt"

    if os.path.exists(best_model):
        validate_model(best_model)
        show_training_plots()
        test_predictions(best_model)

        print("\n" + "=" * 60)
        print("✨ Training Complete!")
        print("=" * 60)
        print(f"\n📁 New model: {best_model}")
        print(f"\n💡 This model is trained on:")
        print("   • Real chess board images")
        print("   • Multiple lighting conditions")
        print("   • Various angles and positions")
        print("   • Should work MUCH better on real boards!")

        print("\n💡 To use:")
        print("   model = YOLO('runs/chess_board_finetuned/exp/weights/best.pt')")
        print("   results = model.predict('board.jpg', conf=0.15)")

        print("\n💡 To download:")
        print("   from google.colab import files")
        print(f"   files.download('{best_model}')")
        print("=" * 60)
    else:
        print(f"\n❌ Model not found: {best_model}")


def download_model():
    """Download fine-tuned model"""
    from google.colab import files

    model_path = "runs/chess_detection/exp/weights/best.pt"
    if os.path.exists(model_path):
        files.download(model_path)
        print("✅ Model downloaded!")
    else:
        print("❌ Model not found!")


if __name__ == "__main__":
    main()

# After training: download_model()

♟️  Real Chess Board Dataset Fine-tuning
⚠️  No GPU! Enable GPU for faster training

🔄 Converting label files...
📊 Found 517 images

📋 Dataset breakdown:
   Single pieces: 438
   Full boards: 79
   Total: 517

🔀 Split: 465 train (90%), 52 val (10%)
  ⚠️  No label for: WhiteBishop.png
  ⚠️  No label for: WhitePawn_2436 2.png
  ✓ Processed 100/517 images
  ✓ Processed 200/517 images
  ✓ Processed 300/517 images
  ✓ Processed 400/517 images
  ⚠️  No label for: FullBoardGame_2546.png
  ⚠️  No label for: FullBoardGame_2590.png
  ⚠️  No label for: Board10.jpeg
  ⚠️  No label for: Board8.jpeg
  ⚠️  No label for: Board5.jpeg
  ⚠️  No label for: Board2.jpeg
  ⚠️  No label for: FullBoardGame_2548.png
  ⚠️  No label for: FullBoardGame_2559.png
  ⚠️  No label for: Board3.jpeg
  ⚠️  No label for: Board1.jpeg
  ⚠️  No label for: FullBoardGame_2567.png
  ⚠️  No label for: FullBoardGame_2572.png
  ⚠️  No label for: FullBoardGame_2575.png
  ⚠️  No label for: Board4.jpeg
  ⚠️  No label for: FullBoardGam

KeyboardInterrupt: 

## "dataset_3"
> Get Dataset in
> https://github.com/daylen/chess-id

In [ ]:
"""
แปลง Chess Classification Dataset (แยก folder ตามชิ้น) เป็น YOLO format
แล้ว Fine-tune model เดิมต่อ
"""

import os
import shutil
from pathlib import Path
from PIL import Image
import random
import yaml
from ultralytics import YOLO
import torch
from IPython.display import Image as IPImage, display

# ===== Extract Dataset File =====
ext_zip("dataset_3.zip", "dataset_3")

# ===== Configuration =====
class Config:
    # ระบุ path ของ dataset ใหม่
    INPUT_DIR = "dataset_3"
    OUTPUT_DIR = "dataset_3_restruct"

    # Folder mapping: ชื่อ folder -> class name ใหม่
    FOLDER_TO_CLASS = {
        # White pieces
        'wr': 'white-rook',
        'wn': 'white-knight',
        'wb': 'white-bishop',
        'wq': 'white-queen',
        'wk': 'white-king',
        'wp': 'white-pawn',

        # Black pieces
        'br': 'black-rook',
        'bn': 'black-knight',
        'bb': 'black-bishop',
        'bq': 'black-queen',
        'bk': 'black-king',
        'bp': 'black-pawn',

        # Empty - ข้ามไป
        'empty': None
    }

    # Class names ตามลำดับที่ต้องการ
    CLASS_NAMES = [
        "black-bishop", "black-king", "black-knight", "black-pawn",
        "black-queen", "black-rook", "white-bishop", "white-king",
        "white-knight", "white-pawn", "white-queen", "white-rook"
    ]

    # Pre-trained model ที่ train ไว้แล้ว
    PRETRAINED_MODEL = "runs/chess_board_finetuned/exp/weights/best.pt"

    # Training
    EPOCHS = 50  # Fine-tune ไม่ต้องเยอะ
    IMG_SIZE = 416  # ลดจาก 640 เพราะรูปต้นฉบับ 227x227 (เร็วกว่า 30%)
    BATCH_SIZE = 32  # เพิ่มได้เพราะรูปเล็ก


def create_yolo_dataset_from_folders():
    """แปลง classification dataset เป็น YOLO detection format"""
    print("🔄 Converting classification dataset to YOLO format...")

    # ลบ output เก่า
    if os.path.exists(Config.OUTPUT_DIR):
        shutil.rmtree(Config.OUTPUT_DIR)

    # สร้าง structure
    for split in ['train', 'val']:
        os.makedirs(f"{Config.OUTPUT_DIR}/images/{split}", exist_ok=True)
        os.makedirs(f"{Config.OUTPUT_DIR}/labels/{split}", exist_ok=True)

    # สร้าง class_id mapping
    class_to_id = {name: idx for idx, name in enumerate(Config.CLASS_NAMES)}

    total_train = 0
    total_val = 0

    # ตรวจสอบ structure ของ dataset
    split_folders = []

    # กรณีที่ 1: มี output_train/output_test
    if os.path.exists(os.path.join(Config.INPUT_DIR, 'output_train')):
        split_folders = ['output_train', 'output_test']
        print("📁 Detected structure: output_train/output_test")

    # กรณีที่ 2: มี train/test
    elif os.path.exists(os.path.join(Config.INPUT_DIR, 'train')):
        split_folders = ['train', 'test']
        print("📁 Detected structure: train/test")

    # กรณีที่ 3: folders อยู่ตรงๆ (wr, wn, bp, etc.)
    else:
        print("📁 Detected structure: flat folders (wr, wn, bp, ...)")
        split_folders = ['.']  # ใช้ current directory

    # วนลูปแต่ละ split
    for split_folder in split_folders:
        if split_folder == '.':
            split_path = Config.INPUT_DIR
        else:
            split_path = os.path.join(Config.INPUT_DIR, split_folder)

        if not os.path.exists(split_path):
            print(f"  ⚠️  {split_folder} not found, skipping...")
            continue

        print(f"\n📦 Processing: {split_folder}")

        # อ่าน folders ในแต่ละ split
        for folder_name in os.listdir(split_path):
            folder_path = os.path.join(split_path, folder_name)

            if not os.path.isdir(folder_path):
                continue

            # แปลงชื่อ folder เป็น class name
            class_name = Config.FOLDER_TO_CLASS.get(folder_name)

            if class_name is None:
                print(f"  ⊘ Skipping folder: {folder_name}")
                continue

            if class_name not in class_to_id:
                print(f"  ⚠️  Unknown class: {class_name}")
                continue

            class_id = class_to_id[class_name]

            # อ่านรูปในแต่ละ folder
            image_files = [f for f in os.listdir(folder_path)
                          if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

            print(f"  📁 {folder_name:8s} → {class_name:15s} (class {class_id}): {len(image_files)} images")

            for img_idx, img_file in enumerate(image_files):
                img_path = os.path.join(folder_path, img_file)

                try:
                    # อ่านรูป
                    img = Image.open(img_path)
                    w, h = img.size

                    # กำหนด split (80% train, 20% val)
                    is_val = random.random() < 0.2
                    split_name = 'val' if is_val else 'train'

                    # สร้างชื่อไฟล์ใหม่
                    new_filename = f"{folder_name}_{img_idx:04d}"

                    # Copy รูป
                    img_output = f"{Config.OUTPUT_DIR}/images/{split_name}/{new_filename}.jpg"
                    img.save(img_output)

                    # สร้าง label (object อยู่กลางรูปพอดี)
                    # Format: class_id center_x center_y width height (normalized 0-1)
                    label_output = f"{Config.OUTPUT_DIR}/labels/{split_name}/{new_filename}.txt"
                    with open(label_output, 'w') as f:
                        # ใช้ bounding box ที่ครอบคลุมเกือบทั้งรูป (90%)
                        center_x = 0.5
                        center_y = 0.5
                        box_w = 0.9
                        box_h = 0.9
                        f.write(f"{class_id} {center_x} {center_y} {box_w} {box_h}\n")

                    if is_val:
                        total_val += 1
                    else:
                        total_train += 1

                except Exception as e:
                    print(f"    ⚠️  Error processing {img_file}: {e}")
                    continue

    print(f"\n✅ Conversion complete!")
    print(f"   Train: {total_train} images")
    print(f"   Val: {total_val} images")

    return total_train > 0


def create_yaml():
    """สร้าง data.yaml"""
    print("\n📝 Creating data.yaml...")

    yaml_content = {
        'path': os.path.abspath(Config.OUTPUT_DIR),
        'train': 'images/train',
        'val': 'images/val',
        'nc': len(Config.CLASS_NAMES),
        'names': Config.CLASS_NAMES
    }

    yaml_path = f"{Config.OUTPUT_DIR}/data.yaml"
    with open(yaml_path, 'w') as f:
        yaml.dump(yaml_content, f, default_flow_style=False, sort_keys=False)

    print(f"✅ Created: {yaml_path}")
    return yaml_path


def fine_tune_model(yaml_path):
    """Fine-tune model เดิมด้วยข้อมูลใหม่"""
    print("\n" + "=" * 60)
    print("🔥 Fine-tuning Model")
    print("=" * 60)

    # Check GPU
    device = 0 if torch.cuda.is_available() else 'cpu'

    if device == 'cpu':
        print("\n⚠️  Training on CPU will be slow!")
        Config.EPOCHS = 20
        Config.BATCH_SIZE = 8
    else:
        print(f"🎮 GPU: {torch.cuda.get_device_name(0)}")

    # โหลด model ที่ train ไว้แล้ว
    if os.path.exists(Config.PRETRAINED_MODEL):
        print(f"\n✅ Loading pre-trained model: {Config.PRETRAINED_MODEL}")
        model = YOLO(Config.PRETRAINED_MODEL)
    else:
        print(f"\n⚠️  Pre-trained model not found!")
        print(f"   Using YOLOv8n instead...")
        model = YOLO('yolov8n.pt')

    print(f"\nTraining settings:")
    print(f"  Epochs: {Config.EPOCHS}")
    print(f"  Batch: {Config.BATCH_SIZE}")
    print(f"  Image size: {Config.IMG_SIZE}")
    print("=" * 60 + "\n")

    # Fine-tune with AGGRESSIVE augmentation (เพราะรูปเป็น crop แล้ว)
    results = model.train(
        data=yaml_path,
        epochs=Config.EPOCHS,
        imgsz=Config.IMG_SIZE,
        batch=Config.BATCH_SIZE,

        # Geometric augmentation (เพิ่มมากเพราะรูป crop แล้ว)
        degrees=25.0,          # หมุนได้เยอะ
        translate=0.2,         # เลื่อนได้เยอะ
        scale=0.7,             # ขยาย/ย่อได้เยอะ
        shear=10.0,            # shear ได้เยอะ
        perspective=0.001,
        flipud=0.5,            # flip ขึ้น-ลงได้ (เพราะเป็นชิ้นเดี่ยว)
        fliplr=0.5,

        # Color augmentation (มากมาก สำหรับหมากดำ)
        hsv_h=0.03,
        hsv_s=0.9,
        hsv_v=0.7,

        # Advanced augmentation
        mosaic=1.0,
        mixup=0.2,
        copy_paste=0.3,

        # Training settings
        patience=30,
        save=True,
        device=device,
        workers=4 if device != 'cpu' else 2,
        project='runs/chess_finetuned',
        name='exp',
        exist_ok=True,
        optimizer='auto',
        verbose=True,
        seed=42,

        # Loss settings (เน้นที่ classification)
        cls=1.0,               # Classification loss สูง
        box=5.0,               # Box loss
        dfl=1.0,

        # Learning rate (ต่ำกว่าปกติเพราะ fine-tune)
        lr0=0.001,             # Initial learning rate ต่ำ
        lrf=0.01,              # Final learning rate

        amp=True if device != 'cpu' else False,
        fraction=1.0,
        cache=False,
    )

    print("\n✅ Fine-tuning completed!")
    return results


def validate_model(model_path):
    """Validate model"""
    print("\n🔍 Validating model...")

    model = YOLO(model_path)
    metrics = model.val()

    print("\n📊 Validation Results:")
    print("=" * 40)
    print(f"  mAP50:     {metrics.box.map50:.4f}")
    print(f"  mAP50-95:  {metrics.box.map:.4f}")
    print(f"  Precision: {metrics.box.mp:.4f}")
    print(f"  Recall:    {metrics.box.mr:.4f}")
    print("=" * 40)

    return metrics


def test_on_cropped_images(model_path):
    """ทดสอบ model กับรูป crop"""
    print("\n🎯 Testing on cropped images...")

    model = YOLO(model_path)

    # ทดสอบกับรูป validation
    val_img_dir = f"{Config.OUTPUT_DIR}/images/val"

    if not os.path.exists(val_img_dir):
        print("  ⚠️  Validation images not found")
        return

    test_imgs = [f for f in os.listdir(val_img_dir) if f.endswith('.jpg')][:5]

    for img_name in test_imgs:
        img_path = os.path.join(val_img_dir, img_name)

        results = model.predict(
            source=img_path,
            save=True,
            conf=0.1,  # threshold ต่ำ
            iou=0.3,
            show_labels=True,
            show_conf=True,
            line_width=2
        )

        # แสดงผล
        save_path = results[0].save_dir
        pred_img = os.path.join(save_path, img_name)

        if os.path.exists(pred_img):
            print(f"\n📸 {img_name}:")
            display(IPImage(filename=pred_img, width=400))


def show_training_plots():
    """แสดง training plots"""
    plots = [
        'runs/chess_finetuned/exp/results.png',
        'runs/chess_finetuned/exp/confusion_matrix.png',
    ]

    print("\n📊 Training Visualizations:")
    for plot in plots:
        if os.path.exists(plot):
            display(IPImage(filename=plot, width=800))


def main():
    """Main execution"""
    print("=" * 60)
    print("♟️  Chess Cropped Dataset → YOLO Fine-tuning")
    print("=" * 60)

    # Check GPU
    if torch.cuda.is_available():
        print(f"🎮 GPU: {torch.cuda.get_device_name(0)}")
    else:
        print("⚠️  No GPU! Enable: Runtime > Change runtime type > GPU")

    print("=" * 60 + "\n")

    # Step 1: Convert dataset
    success = create_yolo_dataset_from_folders()

    if not success:
        print("\n❌ Failed to convert dataset!")
        return

    # Step 2: Create yaml
    yaml_path = create_yaml()

    # Step 3: Fine-tune
    fine_tune_model(yaml_path)

    # Step 4: Validate
    best_model = "runs/chess_finetuned/exp/weights/best.pt"

    if os.path.exists(best_model):
        validate_model(best_model)
        show_training_plots()
        test_on_cropped_images(best_model)

        print("\n" + "=" * 60)
        print("✨ Fine-tuning Complete!")
        print("=" * 60)
        print(f"\n📁 New model: {best_model}")
        print(f"📁 Old model: {Config.PRETRAINED_MODEL}")

        print("\n💡 To use the new model:")
        print("   model = YOLO('runs/chess_finetuned/exp/weights/best.pt')")
        print("   results = model.predict('image.jpg', conf=0.15)")

        print("\n💡 To download:")
        print("   from google.colab import files")
        print(f"   files.download('{best_model}')")
        print("=" * 60)
    else:
        print(f"\n❌ Model not found: {best_model}")


def download_model():
    """Download fine-tuned model"""
    from google.colab import files

    model_path = "runs/chess_finetuned/exp/weights/best.pt"
    if os.path.exists(model_path):
        files.download(model_path)
        print("✅ Model downloaded!")
    else:
        print("❌ Model not found!")


if __name__ == "__main__":
    main()

# After training: download_model()

ModuleNotFoundError: No module named 'ultralytics'

## "dataset_4"
> https://drive.google.com/drive/folders/1C_Sy1ljmAJdO1_Vem5WIRf5GYxkRB1vS

In [1]:
"""
Fine-tune YOLOv8 Step 4: Integrate Dataset 4 (Fixed with Auto-Split)
Dataset: "dataset_4" -> "dataset_4_restruct"
Goal: Fine-tune from Step 3 model and ensure Validation set exists.
"""

import os
import shutil
import yaml
import random
import glob
from ultralytics import YOLO
import torch
from pathlib import Path
from tqdm import tqdm

# ===== Configuration =====
class Config:
    # 📁 Path ของ Dataset 4 (ต้นฉบับ)
    INPUT_DIR = "dataset_4" 
    
    # 📁 โฟลเดอร์ปลายทาง (ที่จะสร้างใหม่)
    OUTPUT_DIR = "dataset_4_restruct"

    # 🎯 Class Target: 12 Class 
    TARGET_CLASS_NAMES = [
        "black-bishop", "black-king", "black-knight", "black-pawn",
        "black-queen", "black-rook", "white-bishop", "white-king",
        "white-knight", "white-pawn", "white-queen", "white-rook"
    ]

    # 🗺️ Mapping Logic: (Roboflow ID -> Our ID)
    # เช็ค dataset_4/data.yaml ให้ชัวร์ว่า ID ตรงตามนี้
    MAPPING = {
        2: 0,   3: 1,   4: 2,   5: 3,   6: 4,   7: 5,   # Black pieces
        8: 6,   9: 7,   10: 8,  11: 9,  12: 10, 13: 11  # White pieces
    }

    # 🤖 Model Path: แก้ไข Path ให้ปลอดภัย (ใช้ / หรือ r"...")
    PRETRAINED_MODEL = "Chess-Trained-Model/0.76_150epochs_best.pt"

    # ⚙️ Training Settings
    EPOCHS = 100        
    IMG_SIZE = 640
    BATCH_SIZE = 16
    LR0 = 0.0005        
    PATIENCE = 50
    VAL_SPLIT_RATIO = 0.2  # ถ้าไม่มี folder val จะแบ่ง train มา 20%


def process_dataset():
    """แปลง Dataset 4 และทำ Auto-Split หากจำเป็น"""
    print(f"🔄 Processing dataset from: {Config.INPUT_DIR}")
    
    # ล้างโฟลเดอร์เก่าถ้ามี
    if os.path.exists(Config.OUTPUT_DIR):
        try:
            shutil.rmtree(Config.OUTPUT_DIR)
        except PermissionError:
            print("❌ Cannot delete old folder. Please close any files using it.")
            return False

    # สร้างโครงสร้างโฟลเดอร์
    for split in ['train', 'val', 'test']:
        os.makedirs(f"{Config.OUTPUT_DIR}/images/{split}", exist_ok=True)
        os.makedirs(f"{Config.OUTPUT_DIR}/labels/{split}", exist_ok=True)

    folder_map = {'train': 'train', 'valid': 'val', 'test': 'test'}
    total_images = 0
    images_count = {'train': 0, 'val': 0, 'test': 0}

    # --- 1. Copy & Remap ---
    for source_split, target_split in folder_map.items():
        # หา path ที่ถูกต้อง (บางที Roboflow ไม่มี subfolder images)
        source_img_dir = os.path.join(Config.INPUT_DIR, source_split, 'images')
        source_lbl_dir = os.path.join(Config.INPUT_DIR, source_split, 'labels')
        
        if not os.path.exists(source_img_dir):
             source_img_dir = os.path.join(Config.INPUT_DIR, source_split)
             source_lbl_dir = os.path.join(Config.INPUT_DIR, source_split)

        if not os.path.exists(source_img_dir):
            continue # ข้ามถ้าไม่มี folder นี้ในต้นฉบับ

        print(f"   📂 Processing {source_split} -> {target_split}...")
        
        # รองรับหลายนามสกุลไฟล์
        image_extensions = ['*.jpg', '*.jpeg', '*.png', '*.bmp']
        image_files = []
        for ext in image_extensions:
            image_files.extend(glob.glob(os.path.join(source_img_dir, ext)))
        
        for img_path in tqdm(image_files, desc=f"Copying {source_split}"):
            img_file = os.path.basename(img_path)
            
            # Copy Image
            shutil.copy(img_path, os.path.join(Config.OUTPUT_DIR, 'images', target_split, img_file))

            # Process Label
            label_file = os.path.splitext(img_file)[0] + ".txt"
            src_lbl_path = os.path.join(source_lbl_dir, label_file)
            
            # สร้างไฟล์ label เสมอ (แม้จะว่างเปล่าก็ต้องสร้างเพื่อบอกว่าไม่มี object)
            new_lines = []
            if os.path.exists(src_lbl_path):
                with open(src_lbl_path, 'r') as f:
                    for line in f:
                        parts = line.strip().split()
                        if len(parts) >= 5:
                            try:
                                cls_id = int(parts[0])
                                # Map ID
                                if cls_id in Config.MAPPING:
                                    new_id = Config.MAPPING[cls_id]
                                    new_lines.append(f"{new_id} {' '.join(parts[1:])}\n")
                            except ValueError:
                                pass
            
            # เขียนไฟล์ Label ใหม่
            with open(os.path.join(Config.OUTPUT_DIR, 'labels', target_split, label_file), 'w') as f:
                f.writelines(new_lines)
            
            total_images += 1
            images_count[target_split] += 1

    # --- 2. Auto-Split Logic (แก้ปัญหา Val ว่าง) ---
    print(f"\n📊 Initial Count: {images_count}")
    
    if images_count['val'] == 0 and images_count['train'] > 0:
        print("⚠️ Warning: No validation images found! Performing Auto-Split...")
        
        train_img_dir = os.path.join(Config.OUTPUT_DIR, 'images', 'train')
        train_lbl_dir = os.path.join(Config.OUTPUT_DIR, 'labels', 'train')
        val_img_dir = os.path.join(Config.OUTPUT_DIR, 'images', 'val')
        val_lbl_dir = os.path.join(Config.OUTPUT_DIR, 'labels', 'val')

        all_train_imgs = glob.glob(os.path.join(train_img_dir, "*"))
        num_to_move = int(len(all_train_imgs) * Config.VAL_SPLIT_RATIO)
        
        files_to_move = random.sample(all_train_imgs, num_to_move)
        
        print(f"   ✂️ Moving {num_to_move} images from Train to Val...")
        for img_path in tqdm(files_to_move, desc="Splitting Data"):
            filename = os.path.basename(img_path)
            label_name = os.path.splitext(filename)[0] + ".txt"
            
            # Move Image
            shutil.move(img_path, os.path.join(val_img_dir, filename))
            
            # Move Label
            src_lbl = os.path.join(train_lbl_dir, label_name)
            if os.path.exists(src_lbl):
                shutil.move(src_lbl, os.path.join(val_lbl_dir, label_name))
        
        print("✅ Auto-Split Completed!")
    
    elif total_images == 0:
        print("❌ Error: No images found in input directory!")
        return False

    return True


def create_yaml():
    """สร้าง data.yaml"""
    yaml_content = {
        'path': os.path.abspath(Config.OUTPUT_DIR),
        'train': 'images/train',
        'val': 'images/val',
        'test': 'images/test',  # อาจจะว่างเปล่าก็ไม่เป็นไรสำหรับ YOLOv8 (test is optional)
        'nc': len(Config.TARGET_CLASS_NAMES),
        'names': Config.TARGET_CLASS_NAMES
    }
    yaml_path = f"{Config.OUTPUT_DIR}/data.yaml"
    with open(yaml_path, 'w') as f:
        yaml.dump(yaml_content, f, sort_keys=False)
    print(f"📝 YAML created at: {yaml_path}")
    return yaml_path


def train_step_4(yaml_path):
    """เริ่มเทรน Step 4"""
    print("\n" + "="*50)
    print("🚀 STARTING FINE-TUNING (STEP 4)")
    print("="*50)

    # Check Model
    if not os.path.exists(Config.PRETRAINED_MODEL):
        print(f"⚠️  Step 3 model NOT found at: {Config.PRETRAINED_MODEL}")
        print(f"   Using 'yolov8n.pt' (Transfer Learning from scratch) instead.")
        model_to_use = 'yolov8n.pt'
    else:
        print(f"✅ Loading Step 3 Weights: {Config.PRETRAINED_MODEL}")
        model_to_use = Config.PRETRAINED_MODEL

    # Detect Device
    device = 'cpu'
    if torch.cuda.is_available():
        device = 0
        print(f"🎮 GPU: {torch.cuda.get_device_name(0)}")
    elif torch.backends.mps.is_available():
        device = 'mps'
        print("🍎 Mac GPU (MPS) Detected")
    
    # Load Model
    model = YOLO(model_to_use)

    # Train
    model.train(
        data=yaml_path,
        epochs=Config.EPOCHS,
        imgsz=Config.IMG_SIZE,
        batch=Config.BATCH_SIZE,
        lr0=Config.LR0,
        lrf=0.01,
        
        # Augmentation Settings
        mosaic=0.5,
        degrees=5.0,
        fliplr=0.0, # Chess board orientation matters (ซ้ายขวามีผลถ้าวางมุมกล้อง fix) แต่ถ้าหมุนกระดานให้ใช้ 0.5
        
        project='runs/chess_finetune_dataset4',
        name='exp_dataset4',
        exist_ok=True,
        device=device,
        patience=Config.PATIENCE,
        save=True,
        
        # ป้องกัน error path ยาวๆ ใน Windows
        cache=False 
    )
    print("\n✅ Step 4 Complete!")
    print(f"   Best Model: runs/chess_finetune_dataset4/exp_dataset4/weights/best.pt")

if __name__ == "__main__":
    if process_dataset():
        yaml_file = create_yaml()
        train_step_4(yaml_file)
    else:
        print("❌ Dataset processing failed. Please check your input folder.")

🔄 Processing dataset from: dataset_4
   📂 Processing train -> train...


Copying train: 100%|██████████| 638/638 [00:00<00:00, 1770.98it/s]



📊 Initial Count: {'train': 638, 'val': 0, 'test': 0}
⚠️ Warning: No validation images found! Performing Auto-Split...
   ✂️ Moving 127 images from Train to Val...


Splitting Data: 100%|██████████| 127/127 [00:00<00:00, 5108.43it/s]


✅ Auto-Split Completed!
📝 YAML created at: dataset_4_restruct/data.yaml

🚀 STARTING FINE-TUNING (STEP 4)
✅ Loading Step 3 Weights: Chess-Trained-Model/0.76_150epochs_best.pt
🍎 Mac GPU (MPS) Detected
Ultralytics 8.3.235 🚀 Python-3.10.19 torch-2.9.1 MPS (Apple M4)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=dataset_4_restruct/data.yaml, degrees=5.0, deterministic=True, device=mps, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=True, fliplr=0.0, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.0005, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=Chess-Trained-Model/0

## "dataset_5"
Plain Board
> https://drive.google.com/drive/folders/1A6GMplBQypDc3jVb2ivKKQZXyFLH9Tb4

In [3]:
"""
Fine-tune YOLOv8 Step 5: Fix Background Noise (Negative Samples)
Goal: เพิ่มรูปกระดานเปล่า (Empty Board) เข้าไปใน Dataset เดิม เพื่อสอนให้ Model ไม่ทายมั่ว
"""

import os
import shutil
import glob
from ultralytics import YOLO
import yaml

# ===== ⚙️ CONFIGURATION (แก้ตรงนี้) =====
class Config:
    # 1. Path ของ Dataset กระดานเปล่าที่คุณเพิ่งโหลดมา (New Data)
    #    (โฟลเดอร์ที่มี data.yaml และโฟลเดอร์ train/valid/test ของ roboflow)
    NEW_EMPTY_BOARD_DIR = "dataset_5"  # <--- แก้ชื่อโฟลเดอร์ให้ตรงกับของคุณ

    # 2. Path ของ Dataset หมากรุกอันเดิมของคุณ (Existing Data)
    #    (อันที่เราใช้เทรนรอบที่แล้ว ที่มี images/train, labels/train)
    EXISTING_DATASET_DIR = "dataset_2_restruct_AJ" # <--- แก้ให้ตรงกับโฟลเดอร์เดิมของคุณ

    # 3. Path ของ Model ตัวล่าสุดที่จะเอามาแก้ (Best Model)
    #    (แก้ Path ให้ชี้ไปที่ไฟล์ best.pt ล่าสุดของคุณ)
    PRETRAINED_MODEL = "runs/chess_board_finetuned_v2/exp_round_3_low_lr/weights/best.pt"

    # 4. Training Settings (เทรนสั้นๆ เบาๆ)
    EPOCHS = 20          # เทรนแค่ 20 รอบพอ (ให้จำ Background ได้ แต่ไม่ลืมของเก่า)
    IMG_SIZE = 640
    BATCH_SIZE = 8       # ลด Batch ลงหน่อยเผื่อรูปเยอะ
    LR0 = 0.0001         # ⚠️ Learning Rate ต่ำๆ (สำคัญมาก! เพื่อไม่ให้ความรู้เดิมหาย)


def add_negative_samples():
    """ดึงรูปกระดานเปล่า -> สร้าง Label ว่าง -> ยัดใส่ Dataset เดิม"""
    print("🚀 เริ่มกระบวนการเพิ่ม Negative Samples...")
    
    # เช็ค Folder
    if not os.path.exists(Config.NEW_EMPTY_BOARD_DIR):
        print(f"❌ ไม่เจอโฟลเดอร์ Dataset ใหม่: {Config.NEW_EMPTY_BOARD_DIR}")
        return False
    if not os.path.exists(Config.EXISTING_DATASET_DIR):
        print(f"❌ ไม่เจอโฟลเดอร์ Dataset เดิม: {Config.EXISTING_DATASET_DIR}")
        return False

    # โฟลเดอร์ปลายทาง (Dataset เดิม)
    dest_img_dir = os.path.join(Config.EXISTING_DATASET_DIR, "images/train")
    dest_lbl_dir = os.path.join(Config.EXISTING_DATASET_DIR, "labels/train")

    count = 0
    
    # วนลูปหาไฟล์รูปในทุกโฟลเดอร์ย่อยของ Dataset ใหม่ (train/valid/test)
    # เราจะเอาทั้งหมดมารวมเป็น Training set เพื่อสอน Background
    for subfolder in ['train', 'valid', 'test', 'val']:
        src_path = os.path.join(Config.NEW_EMPTY_BOARD_DIR, subfolder, 'images')
        
        # รองรับกรณี Roboflow ไม่มี subfolder images (บางทีวางกองรวมกัน)
        if not os.path.exists(src_path):
            src_path = os.path.join(Config.NEW_EMPTY_BOARD_DIR, subfolder)
        
        if not os.path.exists(src_path):
            continue

        # หาไฟล์รูป
        images = glob.glob(os.path.join(src_path, "*.jpg")) + \
                 glob.glob(os.path.join(src_path, "*.png")) + \
                 glob.glob(os.path.join(src_path, "*.jpeg"))

        print(f"   📂 เจอ {len(images)} รูปใน {subfolder}")

        for img_path in images:
            filename = os.path.basename(img_path)
            
            # 1. Copy รูปภาพ ไปใส่ Dataset เดิม
            shutil.copy(img_path, os.path.join(dest_img_dir, filename))

            # 2. สร้างไฟล์ Label .txt "ว่างเปล่า" (Empty File)
            #    (นี่คือหัวใจสำคัญของการทำ Negative Sample)
            lbl_name = os.path.splitext(filename)[0] + ".txt"
            lbl_path = os.path.join(dest_lbl_dir, lbl_name)
            
            with open(lbl_path, 'w') as f:
                pass # ไม่เขียนอะไรเลย = ไม่มีวัตถุ = Background

            count += 1

    print(f"\n✅ เพิ่ม Negative Samples สำเร็จ: {count} รูป")
    print(f"   📍 ไปที่: {dest_img_dir}")
    return count > 0

def train_fix_background():
    """สั่งเทรนต่อเบาๆ"""
    print("\n" + "="*50)
    print("🔥 Start Fine-tuning (Fix Background Noise)")
    print("="*50)

    # เช็คโมเดล
    if not os.path.exists(Config.PRETRAINED_MODEL):
        print(f"❌ ไม่เจอไฟล์โมเดล: {Config.PRETRAINED_MODEL}")
        return

    # โหลดโมเดลเดิม
    model = YOLO(Config.PRETRAINED_MODEL)

    # ไฟล์ data.yaml ของ Dataset เดิม
    yaml_path = os.path.join(Config.EXISTING_DATASET_DIR, "data.yaml")

    # เริ่มเทรน
    model.train(
        data=yaml_path,
        epochs=Config.EPOCHS,
        imgsz=Config.IMG_SIZE,
        batch=Config.BATCH_SIZE,
        
        # Config สำคัญสำหรับการ Fix Background
        lr0=Config.LR0,        # LR ต่ำๆ
        lrf=0.01,
        
        # Augmentation (ปิด Mosaic เพราะเราอยากให้เห็น Background เต็มๆ)
        mosaic=0.0,            
        degrees=0.0,
        
        # Windows Optimization
        workers=0,             # ⚠️ สำคัญ: กัน Error DataLoader
        
        project='runs/chess_finetune_dataset4_fix_bg',
        name='exp_negative_samples',
        exist_ok=True,
        save=True
    )
    
    print("\n✅ Training Complete!")
    print("✨ Model ใหม่ที่ลดหมากผีแล้วอยู่ที่: runs/chess_finetune_dataset4_fix_bg/exp_negative_samples/weights/best.pt")

if __name__ == "__main__":
    # 1. ย้ายรูปและสร้าง Label ว่าง
    if add_negative_samples():
        # 2. ถ้ามีรูปเพิ่มจริง ให้เริ่มเทรน
        train_fix_background()
    else:
        print("⚠️ ไม่มีการเทรนเกิดขึ้น เพราะหารูปไม่เจอ")

🚀 เริ่มกระบวนการเพิ่ม Negative Samples...
   📂 เจอ 10 รูปใน train

✅ เพิ่ม Negative Samples สำเร็จ: 10 รูป
   📍 ไปที่: dataset_2_restruct_AJ/images/train

🔥 Start Fine-tuning (Fix Background Noise)
Ultralytics 8.3.235 🚀 Python-3.10.19 torch-2.9.1 CPU (Apple M4)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=dataset_2_restruct_AJ/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=20, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.0001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=runs/chess_board_fine